# Conditional Monte Carlo

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import time

## Defining Functions

## Black-Scholes Option Pricing

In [2]:
def bsm_price(S, k, r, t, sigma, option='call'):
    '''Calculate Black-Scholes option price'''
    d1 = (np.log(S/k) + (r + 0.5*sigma**2)*t) / (sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    
    if option == 'call':
        price = S*norm.cdf(d1) - k*np.exp(-r*t)*norm.cdf(d2)
    else:
        price = k*np.exp(-r*t)*norm.cdf(-d2) - S*norm.cdf(-d1)
    
    return price

## Black Scholes Vega

In [3]:
def bsm_vega(S, k, r, t, sigma):
    '''Calculate Black-Scholes option vega'''
    d1 = (np.log(S/k) + (r + 0.5*sigma**2)*t) / (sigma*np.sqrt(t))
    vega = S*np.sqrt(t)*norm.pdf(d1)
    return vega

## Implied Volatility Using Newton Rhapson Method

In [4]:
def implied_vol(S, k, r, t, option_price, option='call'):
    '''Calculate implied volatility using the Newton-Raphson method'''
    tol = 1e-5
    max_iter = 10000
    sigma = 0.5
    
    for i in range(max_iter):
        price = bsm_price(S, k, r, t, sigma, option)
        vega = bsm_vega(S, k, r, t, sigma)
        diff = price - option_price
        
        if abs(diff) < tol:
            break
        
        sigma = sigma - diff/vega
    
    return sigma

## Heston Call Option Price

In [5]:
import numpy as np
from scipy.integrate import quad

def HestonCallQuad(kappa, theta, sigma, rho, v0, r, T, s0, K):
    call = s0 * HestonP(kappa, theta, sigma, rho, v0, r, T, s0, K, 1) - \
           K * np.exp(-r * T) * HestonP(kappa, theta, sigma, rho, v0, r, T, s0, K, 2)
    return call

def HestonP(kappa, theta, sigma, rho, v0, r, T, s0, K, type):
    integrand = lambda phi: HestonPIntegrand(phi, kappa, theta, sigma, rho, v0, r, T, s0, K, type)
    ret = 0.5 + (1/np.pi) * quad(integrand, 0, np.inf)[0]
    return ret

def HestonPIntegrand(phi, kappa, theta, sigma, rho, v0, r, T, s0, K, type):
    ret = np.real(np.exp(-1j * phi * np.log(K)) * Hestf(phi, kappa, theta, sigma, rho, v0, r, T, s0, type) / (1j * phi))
    return ret

def Hestf(phi, kappa, theta, sigma, rho, v0, r, T, s0, type):
    if type == 1:
        u = 0.5
        b = kappa - rho * sigma
    else:
        u = -0.5
        b = kappa
    d = np.sqrt((rho * sigma * phi * 1j - b)**2 - (2 * u * phi * 1j - phi**2) * sigma**2)

    g = (b - rho * sigma * phi * 1j - d) / (b - rho * sigma * phi * 1j + d)

    C = (theta * kappa / sigma**2) * ((b - rho * sigma * phi * 1j - d) * T - 2 * np.log((1 - g * np.exp(-d * T)) / (1 - g)))
    D = (1 / sigma**2) * (b - rho * sigma * phi * 1j - d) * ((1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T)))

    f = np.exp(C + D * v0 + 1j * phi * (np.log(s0) + r * T))
    return f

## Initializing Parameters

In [6]:
S0 = 100
r = 0.05
K = np.arange(90, 121,5).reshape(1,7)      #column vector
T = 3/12      # time to maurity
dt = 1/365    #Setting step size delta t = 1/365
steps = int(90)

## CIR Parameters
X0 = 0.2
kappa = 3
theta = 0.2
rho = np.array([0, -0.3, -0.7])
sigma = np.sqrt(2*theta*kappa)

M = 20000 # number of monter carlo simulations
N = steps+1
num_est = 100


## Initializing Necessary Arrays



In [7]:
stock_price = np.zeros((1,M))
strike = np.tile(K,(M,1)) 
call_mc = np.zeros((rho.shape[0], K.shape[1], num_est))
call_cmc = np.zeros((rho.shape[0],K.shape[1], num_est))
discount = math.exp(-r*T)  

imp_vol_mc = np.zeros((rho.shape[0], K.shape[1], num_est))
imp_vol_cmc = np.zeros((rho.shape[0], K.shape[1], num_est))

## Conditional Montecarlo
Z_cm = np.zeros((1,M))
sigma_cm = Z = np.zeros((1,M))


Z_cm = np.zeros((rho.shape[0], num_est, M))
sigma_cm = Z = np.zeros((rho.shape[0], num_est, M))

S_cm = np.zeros(call_cmc.shape)
heston_call = np.zeros((len(rho),K.shape[1]))

price = np.zeros((rho.shape[0],K.shape[1],5))
implied_volatility = np.zeros((rho.shape[0],K.shape[1],5))

## Main Code for MC and CMC

In [8]:
start = time.time()
for i, rh in enumerate(rho):
    for j in range(num_est):
        X_implicit = X0 * np.ones((M,N))
        Y = np.log(S0) * np.ones((M,N))
        dW = np.random.normal(0,1,size = (M,N))*np.sqrt(dt)   # standard brownian motion
        dB = np.random.normal(0,1,size = (M,N))*np.sqrt(dt)     # standard brownian motion    
        for t in range(1,N):
            ## Simulate CIR process using implicit scheme
#             c = -(1 - kappa * dt ) * X_implicit[:,t-1] - (kappa * theta - 0.5 * sigma**2) * dt
#             b = - sigma * dW[:,t]
#             a = 1
#             root = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
#             X_implicit[:,t] = root**2
            
            X_implicit[:,t] = (1 - kappa * dt ) * X_implicit[:,t -1] + kappa * theta * dt + sigma * np.sqrt(np.maximum(X_implicit[:,t -1],0))*dW[:,t]
            Y[:,t] = Y[:,t-1] + ((np.full((M),r) - 0.5 * X_implicit[:,t-1])*dt) + \
                             ((np.sqrt(np.maximum(X_implicit[:,t-1],0))*(rh*dW[:,t] + np.sqrt(1-rh**2) * dB[:,t])))
    
        z = rh * np.sum(np.sqrt(np.maximum(X_implicit,0))*dB, axis = 1) - 0.5*(rh**2)*np.sum(X_implicit*dt, axis = 1)
        sig = np.sqrt((1-rh**2)*np.sum(X_implicit*dt, axis = 1)/T)

        Z_cm[i,j,:] = z
        sigma_cm[i,j,:] = sig 
        
        #np.vstack([sigma_cm,sig.reshape(1,M)])

        S = np.repeat(np.exp(Y[:,-1]).reshape(M,1),K.shape[1],axis = 1)
        call_mc[i,:,j] = discount*np.mean(np.maximum(S - strike,0),axis = 0)
        
# Z_cm = Z_cm[1:,:]
# sigma_cm = sigma_cm[1:,:]
S_cmc = S0*np.exp(Z_cm)
end = time.time()
print("Total time required : ", end-start)

Total time required :  62.72668695449829


## Calculating Call Price Using Conditional MC

In [9]:
for i,k in enumerate(K[0]):
    c = bsm_price(S_cmc, k, r, T, sigma_cm, option='call')
    call_cmc[:,i,:] = np.mean(c, axis = 2)

## Implied Volatility Calculation

In [10]:
for i in range(S_cm.shape[0]):
    S_cm[i,:,:] = np.repeat(np.mean(S_cmc, axis = 2)[i,:].reshape(1,num_est),7, axis = 0)

## Calculating Implied Volatility for MC and CMC using Newton Rahpson Method

for i in range(call_mc.shape[0]):
    for j in range(call_mc.shape[1]):
        for k in range(call_mc.shape[2]):
            imp_vol_mc[i,j,k] = implied_vol(S0,float(K[0,j]),r,T,float(call_mc[i,j,k]),'call')
            
for i in range(call_cmc.shape[0]):
    for j in range(call_cmc.shape[1]):
        for k in range(call_cmc.shape[2]):
            imp_vol_cmc[i,j,k] = implied_vol(S_cm[i,j,k],float(K[0,j]),r,T,float(call_cmc[i,j,k]),'call')

            

## Heston Call Price

In [11]:
for i, rh in enumerate(rho):
    for j, strike in enumerate(K[0]):
        heston_call[i,j] = HestonCallQuad(kappa,theta,sigma, rh, X0, r, T, S0, strike)

## Heston Implied Volatility

In [12]:
hest_impvol = np.zeros((len(rho),K.shape[1]))
for i, rh in enumerate(rho):
    for j, strike in enumerate(K[0]):
        heston_call[i,j] = HestonCallQuad(kappa,theta,sigma, rh, X0, r, T, S0, strike)
for i,rh in enumerate(rho):
    for j, strike in enumerate(K[0]):
        hest_impvol[i,j] = implied_vol(S0, strike, r, T, heston_call[i,j], option='call')

## Converting Numpy Arrays to DataFrames

In [13]:
mc_rho_0 = pd.DataFrame(call_mc[0,:,:], index = ["K = " + str(i) for i in K[0]], columns = range(1,101))
mc_rho_1 = pd.DataFrame(call_mc[1,:,:],index = ["K = " + str(i) for i in K[0]], columns = range(1,101))
mc_rho_2 = pd.DataFrame(call_mc[2,:,:],index = ["K = " + str(i) for i in K[0]], columns = range(1,101))
cmc_rho_0 = pd.DataFrame(call_cmc[0,:,:],index = ["K = " + str(i) for i in K[0]], columns = range(1,101))
cmc_rho_1 = pd.DataFrame(call_cmc[1,:,:],index = ["K = " + str(i) for i in K[0]], columns = range(1,101))
cmc_rho_2 = pd.DataFrame(call_cmc[2,:,:],index = ["K = " + str(i) for i in K[0]], columns = range(1,101))
df_hc_1 = pd.DataFrame(heston_call[0,:], index = ["K = " + str(i) for i in K[0]],columns = ['Heston Call Price'])
df_hc_2 = pd.DataFrame(heston_call[1,:], index = ["K = " + str(i) for i in K[0]],columns = ['Heston Call Price'])
df_hc_3 = pd.DataFrame(heston_call[2,:], index = ["K = " + str(i) for i in K[0]],columns = ['Heston Call Price'])
df_hc_ivol_1 = pd.DataFrame(hest_impvol[0,:], index = ["K = " + str(i) for i in K[0]],columns = ['Heston Implied Volatility'])
df_hc_ivol_2 = pd.DataFrame(hest_impvol[1,:], index = ["K = " + str(i) for i in K[0]],columns = ['Heston Implied Volatility'])
df_hc_ivol_3 = pd.DataFrame(hest_impvol[2,:], index = ["K = " + str(i) for i in K[0]],columns = ['Heston Implied Volatility'])


## Average and SE of MC and CMC prices

In [14]:
price_avg_mc = np.mean(call_mc, axis = 2)       ## 3x7 numpy array
price_avg_cmc = np.mean(call_cmc, axis = 2)    ## 3x7 numpy array
price_se_mc = np.std(call_mc, axis = 2)      ## 3x7 numpy array
price_se_cmc = np.std(call_cmc, axis = 2)     ## 3x7 numpy array

## Average and SE of MC and CMC Implied Vols

In [15]:
imp_vol_avg_mc = np.mean(imp_vol_mc, axis = 2)       ## 3x7 numpy array
imp_vol_avg_cmc = np.mean(imp_vol_cmc, axis = 2)    ## 3x7 numpy array
imp_vol_se_mc = np.std(imp_vol_mc, axis = 2)      ## 3x7 numpy array
imp_vol_se_cmc = np.std(imp_vol_cmc, axis = 2)     ## 3x7 numpy array

## Creating Final Data Array

In [16]:
for i,rh in enumerate(rho):
    price[i,:,0] = heston_call[i,:]
    price[i,:,1] = price_avg_mc[i,:]
    price[i,:,2] = price_avg_cmc[i,:]
    price[i,:,3] = price_se_mc[i,:]
    price[i,:,4] = price_se_cmc[i,:]
    implied_volatility[i,:,0] = hest_impvol[i,:]
    implied_volatility[i,:,1] = imp_vol_avg_mc[i,:]
    implied_volatility[i,:,2] = imp_vol_avg_cmc[i,:]
    implied_volatility[i,:,3] = imp_vol_se_mc[i,:]
    implied_volatility[i,:,4] = imp_vol_se_cmc[i,:]

## Analysis of Price and Implied Volatilities

In [17]:
price_rho_0 = pd.DataFrame(price[0,:,:],index = [str(i) for i in K[0]],columns = ['Heston Price','Avg. MC', 'Avg. CMC', 'Std-err MC', 'Std-err CMC'])
print("Analysis of Prices for Rho = 0")
price_rho_0 = price_rho_0.rename_axis('Strike')
price_rho_0

Analysis of Prices for Rho = 0


,Heston Price,Avg. MC,Avg. CMC,Std-err MC,Std-err CMC
Strike,,,,,
90,14.924981,14.890312,14.918411,0.119018,0.013054
95,11.799417,11.760236,11.792156,0.109791,0.015023
100,9.170275,9.128765,9.162595,0.100110,0.016153
105,7.030752,6.989029,7.022954,0.089567,0.016322
110,5.338805,5.298104,5.331155,0.079628,0.015644
115,4.030416,3.991761,4.023110,0.070847,0.014380
120,3.034559,3.000150,3.027719,0.063279,0.012811


In [18]:
percent_reduction0 = 100 * (price_rho_0['Std-err CMC'] - price_rho_0['Std-err MC'])/price_rho_0['Std-err MC']
percent_reduction0

Strike
90    -89.031995
95    -86.316633
100   -83.864797
105   -81.777127
110   -80.353394
115   -79.702926
120   -79.754430
dtype: float64

## By comparing the standard errors for Monte Carlo and Conditional Monte Carlo, we observe that there is a higher variance reduction for in the money call options and it decreases as the call option becomes out of the money for ρ  = 0. The reduction in standard deviation is ~ 90% for in the money call options and it goes down to ~ 80% for out of the money call options.

In [19]:
price_rho_1 = pd.DataFrame(price[1,:,:],index = [str(i) for i in K[0]],columns = ['Heston Price','Avg. MC', 'Avg. CMC', 'Std-err MC', 'Std-err CMC'])
print("Analysis of Prices for Rho = - 0.3")
price_rho_1 = price_rho_1.rename_axis('Strike')
price_rho_1

Analysis of Prices for Rho = - 0.3


,Heston Price,Avg. MC,Avg. CMC,Std-err MC,Std-err CMC
Strike,,,,,
90,15.123767,15.067219,14.918804,0.107157,0.032862
95,11.899730,11.840692,11.791840,0.098487,0.030214
100,9.135562,9.076150,9.161854,0.089482,0.027479
105,6.850285,6.792506,7.022087,0.080128,0.024670
110,5.030281,4.975909,5.330386,0.070777,0.021821
115,3.631328,3.582841,4.022551,0.061805,0.019013
120,2.588619,2.547189,3.027392,0.053808,0.016347


In [20]:
percent_reduction1 = 100 * (price_rho_1['Std-err CMC'] - price_rho_1['Std-err MC'])/price_rho_1['Std-err MC']
percent_reduction1

Strike
90    -69.332764
95    -69.322121
100   -69.290770
105   -69.212033
110   -69.169864
115   -69.237085
120   -69.620628
dtype: float64

## For ρ = - 0.3, we observe that there is a higher variance reduction for in the money call options and it decreases as the call option becomes out of the money. The reduction in standard deviation is ~ 70% for in the money call options and it goes down to ~ 65% for out of the money call options. Comparing the variance reduction with ρ = 0, we can see that the variance reduction significanly decreases as the |ρ| increases to 0.3.

In [21]:
price_rho_2 = pd.DataFrame(price[2,:,:],index = [str(i) for i in K[0]],columns = ['Heston Price','Avg. MC', 'Avg. CMC', 'Std-err MC', 'Std-err CMC'])
print("Analysis of Prices for Rho = - 0.7")
price_rho_2 = price_rho_2.rename_axis('Strike')
price_rho_2

Analysis of Prices for Rho = - 0.7


,Heston Price,Avg. MC,Avg. CMC,Std-err MC,Std-err CMC
Strike,,,,,
90,15.351648,15.284472,14.913801,0.099519,0.082691
95,12.013042,11.942091,11.787463,0.091644,0.075913
100,9.079658,9.009017,9.158084,0.082148,0.068785
105,6.591095,6.523905,7.018873,0.071522,0.061604
110,4.569949,4.509799,5.327678,0.060868,0.054582
115,3.013378,2.963173,4.020318,0.050878,0.047889
120,1.886748,1.847440,3.025610,0.040437,0.041675


In [22]:
percent_reduction2 = 100 * (price_rho_2['Std-err CMC'] - price_rho_2['Std-err MC'])/price_rho_2['Std-err MC']
percent_reduction2

Strike
90    -16.908760
95    -17.165654
100   -16.266727
105   -13.867143
110   -10.328317
115    -5.875199
120     3.060484
dtype: float64

## For ρ = - 0.7, we observe that there is a higher variance reduction for in the money call options and it decreases as the call option becomes out of the money. The reduction in standard deviation is ~ 16% for in the money call options and it goes down to ~ -5% for out of the money call options. Comparing the variance reduction with ρ = 0 and ρ = - 0.3, we can see that the variance reduction significanly decreases as the |ρ| increases to 0.7. The variance reduction is least when ρ = - 0.7 and for deep out of the money call options we can even observe an increase in variance.

In [23]:
imp_vol_rho_0 = pd.DataFrame(implied_volatility[0,:,:],index = [str(i) for i in K[0]],columns = ['Heston Price','Avg. MC', 'Avg. CMC', 'Std-err MC', 'Std-err CMC'])
print("Analysis of Implied Volatilities for Rho = 0")
imp_vol_rho_0 = imp_vol_rho_0.rename_axis('Strike')
imp_vol_rho_0

Analysis of Implied Volatilities for Rho = 0


,Heston Price,Avg. MC,Avg. CMC,Std-err MC,Std-err CMC
Strike,,,,,
90,0.436333,0.434168,0.435926,0.007381,0.000808
95,0.432836,0.430702,0.432441,0.005973,0.000817
100,0.431385,0.429275,0.430995,0.005088,0.000821
105,0.431819,0.429723,0.431427,0.004499,0.000820
110,0.433848,0.431727,0.433450,0.004148,0.000814
115,0.437127,0.434951,0.436717,0.003982,0.000806
120,0.441312,0.439154,0.440885,0.003961,0.000799


In [24]:
percent_reduction0 = 100 * (imp_vol_rho_0['Std-err CMC'] - imp_vol_rho_0['Std-err MC'])/imp_vol_rho_0['Std-err MC']
percent_reduction0

Strike
90    -89.049799
95    -86.322459
100   -83.864087
105   -81.779032
110   -80.370517
115   -79.750185
120   -79.839587
dtype: float64

## For ρ = 0, the variance reduction in implied volatility is highest for in the money call options and it decreases as the option goes out of the money. The reduction in standard deviation is ~ 90% for in the money call options and it goes down to ~ 80% for out of the money call options.


In [25]:
imp_vol_rho_1 = pd.DataFrame(implied_volatility[1,:,:],index = [str(i) for i in K[0]],columns = ['Heston Price','Avg. MC', 'Avg. CMC', 'Std-err MC', 'Std-err CMC'])
print("Analysis of Implied Volatilities for Rho = - 0.3")
imp_vol_rho_1 = imp_vol_rho_1.rename_axis('Strike')
imp_vol_rho_1

Analysis of Implied Volatilities for Rho = - 0.3


,Heston Price,Avg. MC,Avg. CMC,Std-err MC,Std-err CMC
Strike,,,,,
90,0.448590,0.445100,0.435751,0.006598,0.000813
95,0.438287,0.435077,0.432268,0.005353,0.000814
100,0.429621,0.426601,0.430832,0.004548,0.000814
105,0.422754,0.419850,0.431280,0.004026,0.000812
110,0.417751,0.414903,0.433321,0.003707,0.000808
115,0.414551,0.411773,0.436608,0.003539,0.000805
120,0.412971,0.410265,0.440795,0.003510,0.000802


In [26]:
percent_reduction1 = 100 * (imp_vol_rho_1['Std-err CMC'] - imp_vol_rho_1['Std-err MC'])/imp_vol_rho_1['Std-err MC']
percent_reduction1

Strike
90    -87.670106
95    -84.792801
100   -82.103235
105   -79.835727
110   -78.192632
115   -77.266170
120   -77.158700
dtype: float64

## For ρ = - 0.3, the variance reduction is highest for in the money call options and it decreases as the option goes out of the money similar to the case of rho = 0, however the variance reduction is slightly less for ρ = - 0.3 compared to ρ = 0. The reduction in standard deviation is ~ 88% for in the money call options and it goes down to ~ 75% for out of the money call options.

In [27]:
imp_vol_rho_2 = pd.DataFrame(implied_volatility[2,:,:],index = [str(i) for i in K[0]],columns = ['Heston Price','Avg. MC', 'Avg. CMC', 'Std-err MC', 'Std-err CMC'])
print("Analysis of Implied Volatilities for Rho = - 0.7")
imp_vol_rho_2 = imp_vol_rho_2.rename_axis('Strike')
imp_vol_rho_2

Analysis of Implied Volatilities for Rho = - 0.7


,Heston Price,Avg. MC,Avg. CMC,Std-err MC,Std-err CMC
Strike,,,,,
90,0.462534,0.458425,0.435752,0.006078,0.001251
95,0.444440,0.440586,0.432271,0.004976,0.001268
100,0.426779,0.423190,0.430833,0.004175,0.001307
105,0.409727,0.406349,0.431277,0.003596,0.001360
110,0.393560,0.390379,0.433315,0.003219,0.001427
115,0.378644,0.375660,0.436598,0.003024,0.001504
120,0.365393,0.362573,0.440786,0.002898,0.001591


In [28]:
percent_reduction2 = 100 * (imp_vol_rho_2['Std-err CMC'] - imp_vol_rho_2['Std-err MC'])/imp_vol_rho_2['Std-err MC']
percent_reduction2

Strike
90    -79.417153
95    -74.509226
100   -68.699490
105   -62.171397
110   -55.671612
115   -50.251163
120   -45.104697
dtype: float64

## For ρ = - 0.7, the variance reduction is highest for in the money call options and it decreases as the option goes out of the money similar to the case of ρ = 0 and ρ = - 0.3, however the variance reduction is least for ρ = - 0.7 compared to ρ = 0 and ρ = - 0.3 for a particular strike value. The reduction in standard deviation is ~ 80% for in the money call options and it goes down to ~ 45% for out of the money call options.

## In general, Conditional Monte Carlo method gives a reduced variance for our estimates of option prices and implied volatilities compared to the Standard Monte Carlo method. 

## There is more variance reduction when ρ = 0 as the Brownian motion of stock price and volatility are independent, making the Conditional Monte Carlo method more effective in reducing variance. As |ρ| increase, the brownian motions become more correlated and affects the variance reduction. Also as ρ < 0, the asset price and volatility are negatively correlated so the volatility will be lower when asset price is higher and vice versa. This results in increased variance in estimation of call option price, making it more difficult to achieve variance reduction using Conditional Monte Carlo as |ρ| increases